In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# importing Word2Vec
from gensim.models import Word2Vec
from gensim.models import FastText

In [4]:
df=pd.read_csv('/content/drive/My Drive/MinorProject2/BERT2/new_german.csv')
df

,Unnamed: 0,tweet_id,text,task1,task2,ID
0,0,1133388798925189122,Deutsche rothaarige porno reife deutsche fraue...,NOT,NONE,hasoc_2020_de_2684
1,1,1131117000279961600,"Lehrstück auch, wie in der linken Jammerfemini...",HOF,NONE,hasoc_2020_de_2440
2,2,1127134592517980161,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NOT,NONE,hasoc_2020_de_1042
3,3,1128897106171842560,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NOT,NONE,hasoc_2020_de_774
4,4,1123576753199484928,@Junge_Freiheit Die Inkas hatten sich schon dä...,NOT,NONE,hasoc_2020_de_559
...,...,...,...,...,...,...
3019,2324,1134395448637173761,RT @alfunterwegs70: Sie belügen euch ganz öffe...,HOF,HATE,hasoc_2020_de_1409
3020,2329,1123614787143962626,"@SandraLustig13 Mannomann, dumme, verwöhnte, g...",HOF,HATE,hasoc_2020_de_3096
3021,2340,1131102202762805248,RT @helllud123: @ZDFheute @sven_giegold @Joerg...,HOF,HATE,hasoc_2020_de_177
3022,2348,1124930687084187649,RT @den_tyske: ... gibt es wirklich noch Idiot...,HOF,HATE,hasoc_2020_de_551


In [5]:
X=df[['text','task2']]
X.head()

,text,task2
0,Deutsche rothaarige porno reife deutsche fraue...,NONE
1,"Lehrstück auch, wie in der linken Jammerfemini...",NONE
2,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NONE
3,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NONE
4,@Junge_Freiheit Die Inkas hatten sich schon dä...,NONE


In [6]:
df1=pd.read_csv('/content/drive/My Drive/MinorProject2/BERT2/german_test_1509.csv')
df1

,tweet_id,text,task1,task2,ID
0,1129095874242650112,@derCarsti Boykottieren hört sich besser an. 💙💙,NOT,NONE,hasoc_2020_de_1053
1,1129004308396236800,"RT @ibikus31: Es wird spekuliert, ob Merkel ei...",NOT,NONE,hasoc_2020_de_671
2,1130896929355907080,Hat #Hitler wirklich den Krieg in der Wüste ve...,NOT,NONE,hasoc_2020_de_2977
3,1132251534329307136,RT @Beatrix_vStorch: #May tritt in UK unter Tr...,NOT,NONE,hasoc_2020_de_1746
4,1124941869115498496,@justmeDoro Eher nicht. Das Gänse hauen wieder...,NOT,NONE,hasoc_2020_de_2416
...,...,...,...,...,...
521,1124809878546128897,RT @ChanMachtSo: SCHMERZEN!!!! Au!!! Mein Gehi...,NOT,NONE,hasoc_2020_de_486
522,1132433240000798720,Die Zerstörung der Grünen. https://t.co/SIYDJj...,NOT,NONE,hasoc_2020_de_3388
523,1127366294255357958,"RT @PParzival: ""Antideutsche"" Pseudo-linke Ide...",NOT,NONE,hasoc_2020_de_2745
524,1124362090460975105,Klug reden und dumm leben.,NOT,NONE,hasoc_2020_de_236


In [7]:
Y=df1[['text','task2']]
Y.head(20)

,text,task2
0,@derCarsti Boykottieren hört sich besser an. 💙💙,NONE
1,"RT @ibikus31: Es wird spekuliert, ob Merkel ei...",NONE
2,Hat #Hitler wirklich den Krieg in der Wüste ve...,NONE
3,RT @Beatrix_vStorch: #May tritt in UK unter Tr...,NONE
4,@justmeDoro Eher nicht. Das Gänse hauen wieder...,NONE
5,RT @VonSchwer: Ein guter Patriot geht von Bord...,NONE
6,@yooli_derechte @MontanaBlack Mich wundert es ...,OFFN
7,Da war #SPD #Kuehnert und am Ende auch #Nahles...,NONE
8,"Nazan, komm bittebitte zurück, löse die „Pungg...",NONE
9,RT @Junge_Freiheit: Die europäische Spitzenkan...,NONE


In [8]:
cols=['task2']
X[cols] = X[cols].apply(lambda x: pd.factorize(x)[0] + 0)
Y[cols] = Y[cols].apply(lambda x: pd.factorize(x)[0] + 0)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
X.head()

,text,task2
0,Deutsche rothaarige porno reife deutsche fraue...,0
1,"Lehrstück auch, wie in der linken Jammerfemini...",0
2,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,0
3,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,0
4,@Junge_Freiheit Die Inkas hatten sich schon dä...,0


In [10]:
Y.head()

,text,task2
0,@derCarsti Boykottieren hört sich besser an. 💙💙,0
1,"RT @ibikus31: Es wird spekuliert, ob Merkel ei...",0
2,Hat #Hitler wirklich den Krieg in der Wüste ve...,0
3,RT @Beatrix_vStorch: #May tritt in UK unter Tr...,0
4,@justmeDoro Eher nicht. Das Gänse hauen wieder...,0


In [11]:
print(X['task2'].value_counts())
print(Y['task2'].value_counts())



0    1860
2     406
3     394
1     364
Name: task2, dtype: int64
0    378
2     88
1     36
3     24
Name: task2, dtype: int64


In [12]:
X_train=X['text']
X_train

0       Deutsche rothaarige porno reife deutsche fraue...
1       Lehrstück auch, wie in der linken Jammerfemini...
2       RT @NDRinfo: Die deutsche Klimaaktivistin Luis...
3       @ruhrbahn jeden Morgen eine neue „Fahrzeugstör...
4       @Junge_Freiheit Die Inkas hatten sich schon dä...
                              ...                        
3019    RT @alfunterwegs70: Sie belügen euch ganz öffe...
3020    @SandraLustig13 Mannomann, dumme, verwöhnte, g...
3021    RT @helllud123: @ZDFheute @sven_giegold @Joerg...
3022    RT @den_tyske: ... gibt es wirklich noch Idiot...
3023    RT @sueszling: Ich hasse Deutsche https://t.co...
Name: text, Length: 3024, dtype: object

In [13]:
y_train=X['task2']
y_train

0       0
1       0
2       0
3       0
4       0
       ..
3019    3
3020    3
3021    3
3022    3
3023    3
Name: task2, Length: 3024, dtype: int64

In [14]:
X_test=Y['text']
X_test

0        @derCarsti Boykottieren hört sich besser an. 💙💙
1      RT @ibikus31: Es wird spekuliert, ob Merkel ei...
2      Hat #Hitler wirklich den Krieg in der Wüste ve...
3      RT @Beatrix_vStorch: #May tritt in UK unter Tr...
4      @justmeDoro Eher nicht. Das Gänse hauen wieder...
                             ...                        
521    RT @ChanMachtSo: SCHMERZEN!!!! Au!!! Mein Gehi...
522    Die Zerstörung der Grünen. https://t.co/SIYDJj...
523    RT @PParzival: "Antideutsche" Pseudo-linke Ide...
524                           Klug reden und dumm leben.
525    Wissen wir schon lange....hat das die Merkel e...
Name: text, Length: 526, dtype: object

In [15]:
y_test=Y['task2']
y_test

0      0
1      0
2      0
3      0
4      0
      ..
521    0
522    0
523    0
524    0
525    0
Name: task2, Length: 526, dtype: int64

In [16]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3024,), (526,), (3024,), (526,))

In [17]:
def create_tokens(lines):
  #stop_words = set(nltk.corpus.stopwords.words('english'))
  t_line=lines.apply(lambda x: nltk.word_tokenize(x))
  #t_line=t_line.apply(lambda x: [w for w in x if not w in stop_words])
  return t_line

In [18]:
X_train=create_tokens(X_train)
print(X_train)
tokenizer= Tokenizer()
tokenizer.fit_on_texts(X_train)
tokenizer


0       [Deutsche, rothaarige, porno, reife, deutsche,...
1       [Lehrstück, auch, ,, wie, in, der, linken, Jam...
2       [RT, @, NDRinfo, :, Die, deutsche, Klimaaktivi...
3       [@, ruhrbahn, jeden, Morgen, eine, neue, „Fahr...
4       [@, Junge_Freiheit, Die, Inkas, hatten, sich, ...
                              ...                        
3019    [RT, @, alfunterwegs70, :, Sie, belügen, euch,...
3020    [@, SandraLustig13, Mannomann, ,, dumme, ,, ve...
3021    [RT, @, helllud123, :, @, ZDFheute, @, sven_gi...
3022    [RT, @, den_tyske, :, ..., gibt, es, wirklich,...
3023    [RT, @, sueszling, :, Ich, hasse, Deutsche, ht...
Name: text, Length: 3024, dtype: object


In [19]:
length=np.average(X_train.apply(lambda x:len(x)*len(x)))
length=np.sqrt(length)
print(length)
length=32

19.326695631203073


In [20]:
def encode_text(tokenizer, lines, length):
  encoded = tokenizer.texts_to_sequences(lines)
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [21]:
word_index=tokenizer.word_index

In [22]:
X_test=create_tokens(X_test)
print(X_test)

0      [@, derCarsti, Boykottieren, hört, sich, besse...
1      [RT, @, ibikus31, :, Es, wird, spekuliert, ,, ...
2      [Hat, #, Hitler, wirklich, den, Krieg, in, der...
3      [RT, @, Beatrix_vStorch, :, #, May, tritt, in,...
4      [@, justmeDoro, Eher, nicht, ., Das, Gänse, ha...
                             ...                        
521    [RT, @, ChanMachtSo, :, SCHMERZEN, !, !, !, !,...
522    [Die, Zerstörung, der, Grünen, ., https, :, //...
523    [RT, @, PParzival, :, ``, Antideutsche, '', Ps...
524                   [Klug, reden, und, dumm, leben, .]
525    [Wissen, wir, schon, lange, ..., .hat, das, di...
Name: text, Length: 526, dtype: object


In [23]:
X_train=encode_text(tokenizer,X_train,length)
X_train
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[  22, 3067,   80, ...,    0,    0,    0],
       [3071,   47,    3, ...,    0,    0,    0],
       [   7,    1, 3075, ...,    0,    0,    0],
       ...,
       [   7,    1,  293, ...,    0,    0,    0],
       [   7,    1,  511, ...,    0,    0,    0],
       [   7,    1,  822, ...,    0,    0,    0]], dtype=int32)

In [24]:
X_test=encode_text(tokenizer,X_test,length)
X_test
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[   1,  128,   40, ...,    0,    0,    0],
       [   7,    1,    2, ...,    0,    0,    0],
       [  61,    8,  201, ...,    0,    0,    0],
       ...,
       [   7,    1, 3557, ...,    0,    0,    0],
       [ 861,   10,   46, ...,    0,    0,    0],
       [3549,   79,   34, ...,    0,    0,    0]], dtype=int32)

In [25]:
from keras.utils.np_utils import to_categorical
y_test = to_categorical(np.asarray(y_test))
y_train = to_categorical(np.asarray(y_train))
print(y_test)
print(y_train)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [26]:
# print(len(X_train))
# print(len(X_test))
# print(len(y_train))
# print(len(y_test))
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3024, 32), (526, 32), (3024, 4), (526, 4))

In [27]:
# Creating embedding matrix
# Using 
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/MinorProject/crawl-300d-2M.vec.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [28]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/crawl-300d-2M.vec', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in crawl-300-2M.' % len(embeddings_index))

Total 1999996 word vectors in crawl-300-2M.


In [29]:
print((X_train))
print((X_test))
print((y_train))
print((y_test))

[[  22 3067   80 ...    0    0    0]
 [3071   47    3 ...    0    0    0]
 [   7    1 3075 ...    0    0    0]
 ...
 [   7    1  293 ...    0    0    0]
 [   7    1  511 ...    0    0    0]
 [   7    1  822 ...    0    0    0]]
[[   1  128   40 ...    0    0    0]
 [   7    1    2 ...    0    0    0]
 [  61    8  201 ...    0    0    0]
 ...
 [   7    1 3557 ...    0    0    0]
 [ 861   10   46 ...    0    0    0]
 [3549   79   34 ...    0    0    0]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [30]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(10671, 300)

In [31]:
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Input,LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
model=Sequential()
model.add(Embedding(input_dim = 10671, output_dim = 300, weights=[embedding_matrix], input_length=length,trainable=False))
model.add(LSTM(192, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 300)           3201300   
_________________________________________________________________
lstm (LSTM)                  (None, 192)               378624    
_________________________________________________________________
dense (Dense)                (None, 4)                 772       
Total params: 3,580,696
Trainable params: 379,396
Non-trainable params: 3,201,300
_________________________________________________________________


In [32]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=200,batch_size=256, verbose=2)

Epoch 1/200
12/12 - 2s - loss: 1.1682 - accuracy: 0.5757 - val_loss: 0.9015 - val_accuracy: 0.7186
Epoch 2/200
12/12 - 2s - loss: 1.1062 - accuracy: 0.6154 - val_loss: 0.9384 - val_accuracy: 0.7186
Epoch 3/200
12/12 - 2s - loss: 1.0912 - accuracy: 0.6147 - val_loss: 0.9132 - val_accuracy: 0.7186
Epoch 4/200
12/12 - 2s - loss: 1.0824 - accuracy: 0.6151 - val_loss: 0.9009 - val_accuracy: 0.7186
Epoch 5/200
12/12 - 2s - loss: 1.0785 - accuracy: 0.6151 - val_loss: 0.9006 - val_accuracy: 0.7186
Epoch 6/200
12/12 - 2s - loss: 1.0745 - accuracy: 0.6151 - val_loss: 0.9192 - val_accuracy: 0.7186
Epoch 7/200
12/12 - 2s - loss: 1.0676 - accuracy: 0.6151 - val_loss: 0.9273 - val_accuracy: 0.7186
Epoch 8/200
12/12 - 2s - loss: 1.0473 - accuracy: 0.6154 - val_loss: 0.8931 - val_accuracy: 0.7186
Epoch 9/200
12/12 - 2s - loss: 1.0129 - accuracy: 0.6214 - val_loss: 0.9196 - val_accuracy: 0.7186
Epoch 10/200
12/12 - 2s - loss: 0.9683 - accuracy: 0.6468 - val_loss: 0.8047 - val_accuracy: 0.7186
Epoch 11/

In [33]:
predictions_test = model.predict(X_test)

predictions_test

array([[9.9993944e-01, 5.1463608e-06, 4.0988903e-06, 5.1388248e-05],
       [9.9997449e-01, 7.7381937e-06, 8.5665688e-06, 9.2226237e-06],
       [9.7636777e-01, 5.7106731e-03, 1.7637203e-02, 2.8437245e-04],
       ...,
       [9.9880958e-01, 4.6606274e-06, 1.2959326e-04, 1.0561545e-03],
       [9.9949658e-01, 1.1438036e-04, 1.1808419e-05, 3.7722453e-04],
       [9.9998665e-01, 1.6333386e-06, 5.7857255e-06, 5.8376472e-06]],
      dtype=float32)

In [34]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df1.index)
# output_DataFrame[0]=output_DataFrame[0].map({0:'NGEN',1:'GEN'})
# output_DataFrame.to_csv("German Task2 output.csv")

In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       378
           1       0.15      0.28      0.20        36
           2       0.40      0.07      0.12        88
           3       0.20      0.04      0.07        24

   micro avg       0.71      0.71      0.71       526
   macro avg       0.39      0.33      0.31       526
weighted avg       0.67      0.71      0.67       526
 samples avg       0.71      0.71      0.71       526



In [36]:
# df_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/hin_test.csv")
# df_test.head()

In [37]:
# model.save('HindiTaskB.h5')

In [38]:
# test_text=encode_text(tokenizer,df_test['Text'],length)
# test_text

In [39]:
# predictions_test = model.predict(test_text)

#

In [40]:
# pred_test=np.zeros_like(predictions_test)
# pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
# output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df_test['ID'])
# output_DataFrame[0]=output_DataFrame[0].map({0:'NGEN',1:'GEN'})
# output_DataFrame.to_csv("Hindi Subtask B CNN Fasttext output.csv")

In [41]:
# df_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/hin_test.csv")
# df_test.head()

In [42]:
# test_text=encode_text(tokenizer,df_test['Text'],length)
# test_text

In [43]:
# predictions_test = model.predict(test_text)

# predictions_test

In [44]:
# pred_test=np.zeros_like(predictions_test)
# pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
# output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df_test['ID'])
# output_DataFrame[0]=output_DataFrame[0].map({0:'NGEN',1:'GEN'})
# output_DataFrame.to_csv("Hindi Subtask B LSTM Fasttext output.csv")

In [45]:
  df3=df1[['tweet_id','ID']]
  df3

,tweet_id,ID
0,1129095874242650112,hasoc_2020_de_1053
1,1129004308396236800,hasoc_2020_de_671
2,1130896929355907080,hasoc_2020_de_2977
3,1132251534329307136,hasoc_2020_de_1746
4,1124941869115498496,hasoc_2020_de_2416
...,...,...
521,1124809878546128897,hasoc_2020_de_486
522,1132433240000798720,hasoc_2020_de_3388
523,1127366294255357958,hasoc_2020_de_2745
524,1124362090460975105,hasoc_2020_de_236


In [46]:
predictions_test1=predictions_test.argmax(1)
pred_test1=pd.Series(predictions_test1)
pred_test1

0      0
1      0
2      0
3      0
4      0
      ..
521    3
522    0
523    0
524    0
525    0
Length: 526, dtype: int64

In [47]:
df3['label']=pred_test1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
df3

,tweet_id,ID,label
0,1129095874242650112,hasoc_2020_de_1053,0
1,1129004308396236800,hasoc_2020_de_671,0
2,1130896929355907080,hasoc_2020_de_2977,0
3,1132251534329307136,hasoc_2020_de_1746,0
4,1124941869115498496,hasoc_2020_de_2416,0
...,...,...,...
521,1124809878546128897,hasoc_2020_de_486,3
522,1132433240000798720,hasoc_2020_de_3388,0
523,1127366294255357958,hasoc_2020_de_2745,0
524,1124362090460975105,hasoc_2020_de_236,0


In [49]:
df3['label'] = df3['label'].map({0: "NONE",1:"OFFN",2:"PRFN",3:"HATE"})
df3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet_id,ID,label
0,1129095874242650112,hasoc_2020_de_1053,NONE
1,1129004308396236800,hasoc_2020_de_671,NONE
2,1130896929355907080,hasoc_2020_de_2977,NONE
3,1132251534329307136,hasoc_2020_de_1746,NONE
4,1124941869115498496,hasoc_2020_de_2416,NONE
...,...,...,...
521,1124809878546128897,hasoc_2020_de_486,HATE
522,1132433240000798720,hasoc_2020_de_3388,NONE
523,1127366294255357958,hasoc_2020_de_2745,NONE
524,1124362090460975105,hasoc_2020_de_236,NONE


In [50]:
df3.to_csv('submission_DE_B.csv')

In [52]:
model.save("GermanDatasetSubTask2_withLSTM_Fasttext.h5")